<a href="https://colab.research.google.com/github/gabyrr/CursoTIA/blob/main/TIA_ejemplo_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prediciendo quien sobrevive al desastre del Titánic
Notebook perteneciente al curso: Tópicos de inteligencia artificial: de métodos clásicos a modelos generativos. Por Gabriela Ramírez de la Rosa

In [1]:
# pandas
import pandas as pd
from pandas import Series,DataFrame

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

# machine learning
from sklearn.model_selection import cross_val_predict
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn import feature_selection
from sklearn.preprocessing import LabelEncoder
import sklearn

La biblioteca pandas tiene varios métodos que nos permiten leer archivos en formatos CSV, TSV.

In [3]:
#Método que nos permite leer un archivo en formato CSV
titanic_data= pd.read_csv('TITANIC.csv')
titanic_data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [4]:
#Conociendo la estructura de la tabla
titanic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB


In [ ]:
#Qué tamaño tiene
titanic_data.shape

In [5]:
##Vamos a eliminar las columnas que no nos interesan, al parámetro axis refiere a qué dimensión se va a considerar
#+------------+---------+--------+
#|            |  A      |  B     |
#+------------+---------+---------
#|      0     | 0.626386| 1.52325|----axis=1----->
#+------------+---------+--------+
#             |         |
#             | axis=0  |
#             ↓         ↓

titanic_data = titanic_data.drop(['name','ticket','cabin','boat','body','home.dest'], axis=1)
titanic_data.shape

(1309, 8)

In [ ]:
titanic_data.head()

In [6]:
# get the number of missing data points per column
missing_values_count = titanic_data.isnull().sum()
missing_values_count[0:]

,0
pclass,0
survived,0
sex,0
age,263
sibsp,0
parch,0
fare,1
embarked,2


In [7]:
## cuántos valores diferentes hay en la columna embarked?
titanic_data.embarked.value_counts()

,count
embarked,
S,914
C,270
Q,123


Vamos a rellenar algunos datos faltantes en la columna `embarked`. Lo haremos con el valor más comun en los datos

In [ ]:
titanic_data["embarked"] = titanic_data["embarked"].fillna("S")


In [8]:
## titanic_data['embarked'].value_counts()
## la linea anterior hace lo mimo que la siguiente:
titanic_data.embarked.value_counts()

,count
embarked,
S,914
C,270
Q,123


In [9]:
t_data=titanic_data
titanic_data.head()

,pclass,survived,sex,age,sibsp,parch,fare,embarked
0,1,1,female,29.0000,0,0,211.3375,S
1,1,1,male,0.9167,1,2,151.5500,S
2,1,0,female,2.0000,1,2,151.5500,S
3,1,0,male,30.0000,1,2,151.5500,S
4,1,0,female,25.0000,1,2,151.5500,S


Vamos a cambiar los valores de la variable `embarked` por números

In [10]:
#Emarked refiere al puerto de embarcación C = Cherbourg, Q = Queenstown, S = Southampton
titanic_data.embarked = titanic_data.embarked.map({'C':0, 'Q':1, 'S':2})
titanic_data.head()

,pclass,survived,sex,age,sibsp,parch,fare,embarked
0,1,1,female,29.0000,0,0,211.3375,2.0
1,1,1,male,0.9167,1,2,151.5500,2.0
2,1,0,female,2.0000,1,2,151.5500,2.0
3,1,0,male,30.0000,1,2,151.5500,2.0
4,1,0,female,25.0000,1,2,151.5500,2.0


Hacemos algo parecido para la columna `Fare`, que tiene un renglón sin dato.

Ahora usaremos la media de los valores que sí existen para llenar este hueco. A estos procesos se les denomina operaciones de imputación, la media, o el valor más comun son técnicas tradicionales de imputacion

In [11]:
titanic_data["fare"].fillna(titanic_data["fare"].median(), inplace=True)

Como la variable Fare tiene datos flotantes, haremos una *normalización* de los valores en esta variable

In [12]:
titanic_data.fare.value_counts()

,count
fare,
8.0500,60
13.0000,59
7.7500,55
26.0000,50
7.8958,49
...,...
15.0500,1
9.6875,1
15.5792,1


In [13]:
#Hacemos una normalización de los datos de fare (OJO, no es la mejor técnica)
titanic_data.loc[ titanic_data['fare'] <= 7.91, 'fare'] = 0
titanic_data.loc[(titanic_data['fare'] > 7.91) & (titanic_data['fare'] <= 14.454), 'fare'] = 1
titanic_data.loc[(titanic_data['fare'] > 14.454) & (titanic_data['fare'] <= 31), 'fare'] = 2
titanic_data.loc[ titanic_data['fare'] > 31, 'fare'] = 3

In [14]:
#Decimos que será un valor entero y no un flotante
titanic_data['fare'] = titanic_data['fare'].astype(int)

Ahora rellenaremos los huecos que hacen falta en el atributo de edad. Lo siguiente obtiene el valor medio de edad para los pasajeros de la misma clase

In [15]:
titanic_data['age'] = titanic_data.groupby(['pclass'])['age'].transform(lambda x: x.fillna(x.mean()))

Hacemos enteros los datos de Edad y luego los normalizamos

In [16]:
#primero decimos que serán datos enteros
titanic_data['age'] = titanic_data['age'].astype(int)

#normalizamos en cinco grupos de edades
titanic_data.loc[ titanic_data['age'] <= 16, 'age'] = 0
titanic_data.loc[(titanic_data['age'] > 16) & (titanic_data['age'] <= 32), 'age'] = 1
titanic_data.loc[(titanic_data['age'] > 32) & (titanic_data['age'] <= 48), 'age'] = 2
titanic_data.loc[(titanic_data['age'] > 48) & (titanic_data['age'] <= 64), 'age'] = 3
titanic_data.loc[(titanic_data['age'] > 64), 'age'] = 4


Normalizando datos de `Familia`.
En lugar de tener dos columnas referentes a datos familiares `Parch` & `SibSp`:
- `sibsp` Number of Siblings/Spouses Aboard
- `parch` Number of Parents/Children Aboard
solo se creará una sola columna que represente si el pasajero tenía familiares abordo o no

In [17]:
# Normalizando datos de Familia
titanic_data['family'] =  titanic_data["parch"] + titanic_data["sibsp"]
titanic_data['family'].loc[titanic_data['family'] > 0] = 1
titanic_data['family'].loc[titanic_data['family'] == 0] = 0

<ipython-input-17-b8f070169aa6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titanic_data['family'].loc[titanic_data['family'] > 0] = 1
<ipython-input-17-b8f070169aa6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titanic_data['family'].loc[titanic_data['family'] == 0] = 0


In [18]:
titanic_data.head()

,pclass,survived,sex,age,sibsp,parch,fare,embarked,family
0,1,1,female,1,0,0,3,2.0,0
1,1,1,male,0,1,2,3,2.0,1
2,1,0,female,0,1,2,3,2.0,1
3,1,0,male,1,1,2,3,2.0,1
4,1,0,female,1,1,2,3,2.0,1


In [19]:
#Eliminamos las columnas de parch, sibsp
titanic_data = titanic_data.drop(['sibsp','parch'], axis=1)

Solo falta convertir los valores de la columna `Sex` de categóricos a numéricos

In [20]:
titanic_data['sex'] = titanic_data.sex.map({'female':0, 'male':1})

In [ ]:
titanic_data.head(20)

In [21]:
titanic_data.info()
titanic_data.to_csv("titanic_limpio.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   int64  
 1   survived  1309 non-null   int64  
 2   sex       1309 non-null   int64  
 3   age       1309 non-null   int64  
 4   fare      1309 non-null   int64  
 5   embarked  1307 non-null   float64
 6   family    1309 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 71.7 KB


### Ahora si, definimos nuestras estructuras 'X' y 'y'


In [40]:
X = titanic_data.drop("survived",axis=1)
y = titanic_data["survived"]

### Hagamos pruebas de clasificación

In [23]:
# Import datasets, classifiers and performance metrics
from sklearn import datasets, metrics, svm
from sklearn.model_selection import train_test_split

##Support Vector Machines
lsvm = LinearSVC()
## Naive BAyes
nb = MultinomialNB()
##DecisionTrees
dtree = DecisionTreeClassifier(criterion='entropy')
##Neural Networks
mlp = MLPClassifier()


In [24]:
##split en train y test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False)

In [25]:
clf = dtree

clf.fit(X_train, y_train)

# Predict the value of the digit on the test subset
y_pred = clf.predict(X_test)

In [26]:
print(
    f"Classification report for classifier {clf}:\n"
    f"{metrics.classification_report(y_test, y_pred)}\n"
)

Classification report for classifier DecisionTreeClassifier(criterion='entropy'):
              precision    recall  f1-score   support

           0       0.83      0.88      0.85       207
           1       0.40      0.31      0.35        55

    accuracy                           0.76       262
   macro avg       0.62      0.59      0.60       262
weighted avg       0.74      0.76      0.75       262




## Práctica
* Genere un archivo en excel que nos permita comparar el comportamiento de varios clasificadores, usando un esquema de *validación cruzada a 5 pliegues*. El objetivo es poder determinar qué clasificador se vuelve más efectivo en la predicción.

* Además de la tabla, utilice una gráfica para mostrar un resumen del comportamiento de los distintos clasificadores. Utilice sólo un tipo de gráfica para reportar sus resultados.

In [42]:
## convertir pandaframe a matrix
X = X.to_numpy()
y = y.to_numpy()

In [47]:
from sklearn.model_selection import StratifiedKFold

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=142,
                                                    shuffle=True,
                                                    stratify=y)

cv = StratifiedKFold(n_splits=5, random_state=123, shuffle=True)

## cambiar esta linea para usar otro algoritmo
clf = DecisionTreeClassifier(random_state=123, max_depth=3)


kfold_f = 0.
for train_idx, validacion_idx in cv.split(X_train, y_train):
    #print(validacion_idx)
    clf.fit(X_train[train_idx], y_train[train_idx])
    y_pred = clf.predict(X_train[validacion_idx])
    fscore = metrics.f1_score(y_train[validacion_idx], y_pred, average='macro')
    kfold_f += fscore
kfold_f /= 5

clf = DecisionTreeClassifier(random_state=123, max_depth=3).fit(X_train, y_train)
y_pred = clf.predict(X_test)
test_f = metrics.f1_score(y_test, y_pred, average='macro')

print('Kfold F-score: %.2f%%' % kfold_f)
print('Test F-score: %.2f%%' % test_f)

Kfold F-score: 0.76%
Test F-score: 0.80%
